### Fix-Point Arithmetric


In [1]:
import torch
import syft as sy

In [2]:
# 创建工作机
hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [3]:
# 数据
A = torch.rand(3, 4)
B = torch.rand(4, 5)
print(A, '\n', B)

tensor([[0.0315, 0.9668, 0.2177, 0.5811],
        [0.9274, 0.1507, 0.0476, 0.5183],
        [0.9848, 0.4558, 0.4875, 0.0416]]) 
 tensor([[0.7034, 0.0083, 0.1034, 0.2801, 0.2606],
        [0.4430, 0.0607, 0.5377, 0.5097, 0.4376],
        [0.8103, 0.3309, 0.5820, 0.4222, 0.9640],
        [0.6722, 0.6349, 0.3228, 0.3550, 0.7398]])


In [5]:
# 将数据采用固定精度表示（3位小数），并分发给alice和bob
# fix_precision()方法定义在/root/AriaNN/PySyft/syft/frameworks/torch/tensors/interpreters/native.py line 971
A_ = A.fix_precision(precision_fractional=4).share(bob, alice, crypto_provider=crypto_provider)
B_ = B.fix_precision(precision_fractional=4).share(bob, alice, crypto_provider=crypto_provider)
print(A_, "\n", B_)

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:16413372396 -> bob:39215103197]
	-> [PointerTensor | me:44131232347 -> alice:46363112769]
	*crypto provider: crypto_provider* 
 (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:78935170261 -> bob:94796737335]
	-> [PointerTensor | me:25005339754 -> alice:48700467617]
	*crypto provider: crypto_provider*


In [6]:
C_ = A_.matmul(B_)
C_

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:28956298282 -> bob:51118704681]
	-> [PointerTensor | me:87156791146 -> alice:92208606253]
	*crypto provider: crypto_provider*

In [7]:
C = C_.get().float_precision()
C 

tensor([[1.0172, 0.4997, 0.8372, 0.7996, 1.0708],
        [1.1059, 0.3615, 0.3718, 0.5405, 0.7367],
        [1.3175, 0.2236, 0.6441, 0.7286, 0.9568]])

In [8]:
print(torch.mm(A, B))

tensor([[1.0174, 0.4999, 0.8374, 0.7997, 1.0710],
        [1.1060, 0.3616, 0.3719, 0.5407, 0.7369],
        [1.3177, 0.2236, 0.6442, 0.7288, 0.9569]])
